In [89]:
from langchain import PromptTemplate, OpenAI, LLMChain
import pandas as pd
from config import *
from langchain.text_splitter import RecursiveCharacterTextSplitter
import re

In [4]:
df = pd.read_csv(data_path)

In [23]:
df['CONTENT_LN'] = df.apply(lambda row: len(row['RAW_CONTENT']), axis=1)

In [24]:
df.head()

,ID,RAW_CONTENT,MEETING_NAME,TITLE,PUBLISH_DATE,CONTENT_LN
0,m-1608357555-1682084460,中共中央总书记、国家主席、中央军委主席、中央全面深化改革委员会主任习近平4月21日下午主...,中央全面深化改革会议,会议审议通过了关于强化企业科技创新主体地位的意见关于加强和改进国有经济管理有力支持中国式现代...,2023-04-21T21:41:00Z,1747
1,m-158629831-1682664300,中共中央政治局4月28日召开会议，分析研究当前经济形势和经济工作。中共中央总书记习近平主...,政治局会议,分析研究当前经济形势和经济工作,2023-04-28T14:45:00Z,1440
2,m-704972677-1680162720,中共中央政治局3月30日召开会议，决定从今年4月开始，在全党自上而下分两批开展学习贯彻习...,政治局会议,决定从2023年4月开始在全党自上而下分两批开展学习贯彻习近平新时代中国特色社会主义思想主题...,2023-03-30T15:52:00Z,1653
3,m-248496144-1366366560,中共中央政治局4月19日召开会议，决定从今年下半年开始，用一年左右时间，在全党自上而下分...,政治局会议,研究部署在全党深入开展党的群众路线教育实践活动,2013-04-19T18:16:00Z,1388
4,m-1020277137-1469618520,李克强主持召开国务院常务会议\r\n听取关于地方和部门推进重大项目落地审计情况汇报 完善奖惩...,国务院常委会,李克强主持召开国务院常务会议通过十三五国家科技创新专项规划 以创新型国家建设引领和支撑升级发...,2016-07-27T19:22:00Z,1881


In [26]:
max(df['CONTENT_LN'])

10077

In [35]:
df['CONTENT_LN'].tolist().index(10077)

524

In [5]:
prompt ="""You are given the task to classify a text into a pre-defined topic. You will be given the title and content of the text.
        Here is the list of pre-defined topics: {topic_pdf}.
        Here is the title: {title}.
        Here is the content: {content}.
        Your answer should contain the topic only.
        """

In [54]:
prompt_summary = """
You are given the task to summarize a text. Your summarization cannot be more than 1000 tokens.
Here is the text content: {content}.
Your answer should contain the summary only.
"""

In [36]:
title = df['TITLE'][524]

In [37]:
content = df['RAW_CONTENT'][524]

In [38]:
content[:3000]

'\u3000\u3000各位代表：\r\n\u3000\u3000现在，我代表国务院，向大会作政府工作报告，请予审议，并请全国政协各位委员提出意见。\r\n\u3000\u3000一、过去一年工作回顾\r\n\u3000\u30002004年，全国各族人民在中国共产党领导下，高举邓小平理论和“三个代表”重要思想伟大旗帜，团结奋进，锐意创新，社会主义现代化事业取得令人鼓舞的重大成就。\r\n\u3000\u3000过去一年，我们是在应对新的挑战和考验中前进的。近两年我国经济运行中出现了一些新问题，主要是粮食供求关系趋紧，固定资产投资膨胀，货币信贷投放过快，煤电油运紧张。如果任其发展下去，局部性问题就会演变为全局性问题。党中央、国务院审时度势，及时作出了加强宏观调控的决策和部署，按照果断有力、适时适度、区别对待、注重实效的原则，综合运用经济、法律手段和必要的行政手段，着力解决影响经济平稳较快发展的突出问题。经过全国上下共同努力，宏观调控取得明显成效。经济运行中不稳定不健康因素得到抑制，薄弱环节得到加强，避免了经济大的起落。\r\n\u3000\u3000一年来的成就集中表现在：经济保持平稳较快发展，综合国力进一步增强；改革取得重要进展，对外开放实现新突破；社会事业加快发展，人民生活继续改善。2004年，国内生产总值达到13．65万亿元，比上年增长9．5％；财政收入2．63万亿元，增长21．4％；社会消费品零售总额5．4万亿元，增长13．3％；进出口贸易总额1．15万亿美元，增长35．7％，由上年居世界第四位上升为第三位；城镇新增就业980万人，超过预期目标；城镇居民人均可支配收入9422元，农村居民人均纯收入2936元，扣除价格因素，分别实际增长7．7％和6．8％。这些成就，标志着我国在全面建设小康社会道路上又迈出坚实的一步。\r\n\u3000\u3000一年来，我们主要做了以下几方面工作。\r\n\u3000\u3000（一）采取更直接、更有力的政策措施，促进粮食增产和农民增收。\r\n\u3000\u3000这次宏观调控是从加强农业入手的。主要措施是：减免农业税，取消除烟叶以外的农业特产税，对种粮农民实行直接补贴，对部分地区农民实行良种补贴和农机具购置补贴，对重点粮食品种实行最低收购价政策。大幅度增加对农业、农村特别是粮食主产区的建设投入。全年中央财政用于“三农

In [39]:
print(content)

　　各位代表：
　　现在，我代表国务院，向大会作政府工作报告，请予审议，并请全国政协各位委员提出意见。
　　一、过去一年工作回顾
　　2004年，全国各族人民在中国共产党领导下，高举邓小平理论和“三个代表”重要思想伟大旗帜，团结奋进，锐意创新，社会主义现代化事业取得令人鼓舞的重大成就。
　　过去一年，我们是在应对新的挑战和考验中前进的。近两年我国经济运行中出现了一些新问题，主要是粮食供求关系趋紧，固定资产投资膨胀，货币信贷投放过快，煤电油运紧张。如果任其发展下去，局部性问题就会演变为全局性问题。党中央、国务院审时度势，及时作出了加强宏观调控的决策和部署，按照果断有力、适时适度、区别对待、注重实效的原则，综合运用经济、法律手段和必要的行政手段，着力解决影响经济平稳较快发展的突出问题。经过全国上下共同努力，宏观调控取得明显成效。经济运行中不稳定不健康因素得到抑制，薄弱环节得到加强，避免了经济大的起落。
　　一年来的成就集中表现在：经济保持平稳较快发展，综合国力进一步增强；改革取得重要进展，对外开放实现新突破；社会事业加快发展，人民生活继续改善。2004年，国内生产总值达到13．65万亿元，比上年增长9．5％；财政收入2．63万亿元，增长21．4％；社会消费品零售总额5．4万亿元，增长13．3％；进出口贸易总额1．15万亿美元，增长35．7％，由上年居世界第四位上升为第三位；城镇新增就业980万人，超过预期目标；城镇居民人均可支配收入9422元，农村居民人均纯收入2936元，扣除价格因素，分别实际增长7．7％和6．8％。这些成就，标志着我国在全面建设小康社会道路上又迈出坚实的一步。
　　一年来，我们主要做了以下几方面工作。
　　（一）采取更直接、更有力的政策措施，促进粮食增产和农民增收。
　　这次宏观调控是从加强农业入手的。主要措施是：减免农业税，取消除烟叶以外的农业特产税，对种粮农民实行直接补贴，对部分地区农民实行良种补贴和农机具购置补贴，对重点粮食品种实行最低收购价政策。大幅度增加对农业、农村特别是粮食主产区的建设投入。全年中央财政用于“三农”的支出共2626亿元，增长22．5％。同时，推进农业结构调整，多渠道增加农民收入。这些政策措施力度之大，农民得到实惠之多，是多年来少有的，极大地调动了广大农民的积极性。粮食生产出现重要转机，全年粮食总产量达到46947万吨，增产

In [44]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=50)
texts = text_splitter.split_text(content)

In [46]:
len(texts)

6

In [48]:
print(texts[0])

各位代表：
　　现在，我代表国务院，向大会作政府工作报告，请予审议，并请全国政协各位委员提出意见。
　　一、过去一年工作回顾
　　2004年，全国各族人民在中国共产党领导下，高举邓小平理论和“三个代表”重要思想伟大旗帜，团结奋进，锐意创新，社会主义现代化事业取得令人鼓舞的重大成就。
　　过去一年，我们是在应对新的挑战和考验中前进的。近两年我国经济运行中出现了一些新问题，主要是粮食供求关系趋紧，固定资产投资膨胀，货币信贷投放过快，煤电油运紧张。如果任其发展下去，局部性问题就会演变为全局性问题。党中央、国务院审时度势，及时作出了加强宏观调控的决策和部署，按照果断有力、适时适度、区别对待、注重实效的原则，综合运用经济、法律手段和必要的行政手段，着力解决影响经济平稳较快发展的突出问题。经过全国上下共同努力，宏观调控取得明显成效。经济运行中不稳定不健康因素得到抑制，薄弱环节得到加强，避免了经济大的起落。
　　一年来的成就集中表现在：经济保持平稳较快发展，综合国力进一步增强；改革取得重要进展，对外开放实现新突破；社会事业加快发展，人民生活继续改善。2004年，国内生产总值达到13．65万亿元，比上年增长9．5％；财政收入2．63万亿元，增长21．4％；社会消费品零售总额5．4万亿元，增长13．3％；进出口贸易总额1．15万亿美元，增长35．7％，由上年居世界第四位上升为第三位；城镇新增就业980万人，超过预期目标；城镇居民人均可支配收入9422元，农村居民人均纯收入2936元，扣除价格因素，分别实际增长7．7％和6．8％。这些成就，标志着我国在全面建设小康社会道路上又迈出坚实的一步。
　　一年来，我们主要做了以下几方面工作。
　　（一）采取更直接、更有力的政策措施，促进粮食增产和农民增收。
　　这次宏观调控是从加强农业入手的。主要措施是：减免农业税，取消除烟叶以外的农业特产税，对种粮农民实行直接补贴，对部分地区农民实行良种补贴和农机具购置补贴，对重点粮食品种实行最低收购价政策。大幅度增加对农业、农村特别是粮食主产区的建设投入。全年中央财政用于“三农”的支出共2626亿元，增长22．5％。同时，推进农业结构调整，多渠道增加农民收入。这些政策措施力度之大，农民得到实惠之多，是多年来少有的，极大地调动了广大农民的积极性。粮食生产出现重要转机，全年粮食总产量达到46947万吨，增产38

In [ ]:
llm = OpenAI(temperature = 0, openai_api_key = OPENAI_API_KEY, model_name = 'gpt-4')

In [31]:
llm_chain = LLMChain(
    llm = llm,
    prompt = PromptTemplate.from_template(prompt)
)

In [55]:
llm_chain_summary = LLMChain(
    llm = llm,
    prompt = PromptTemplate.from_template(prompt_summary)
)

In [51]:
llm_chain.predict(topic_pdf = topics_pdf, title = title, content = content[:5000])

'宏观'

In [59]:
summary = []
for x in texts:
    summary.append(llm_chain_summary.predict(content = x))
llm_chain.predict(topic_pdf = topics_pdf, title = title, content = '\n'.join(summary))

'宏观'

In [62]:
def get_topic(title, content):
    texts = text_splitter.split_text(content)
    summary = []
    for x in texts:
        summary.append(llm_chain_summary.predict(content = x))
    return llm_chain.predict(topic_pdf = topics_pdf, title = title, content = '\n'.join(summary))

In [60]:
summary

['在2004年的政府工作报告中，中国政府总结了过去一年的工作成就和挑战。报告强调了在中国共产党的领导下，全国各族人民团结奋进，社会主义现代化事业取得了重大成就。报告指出，面对新的挑战和考验，政府采取了一系列宏观调控措施，包括加强农业，控制固定资产投资过快增长，加强薄弱环节等。这些措施取得了明显成效，经济运行中不稳定不健康因素得到抑制，避免了经济大的起落。报告还强调了推进经济体制改革，扩大对外开放的重要性。总的来说，2004年，中国经济保持平稳较快发展，改革取得重要进展，对外开放实现新突破，社会事业加快发展，人民生活继续改善。',
 '中国政府承诺履行加入世贸组织的义务，降低关税，扩大开放领域，并全面放开外贸经营权。政府加大对科技、教育、文化、卫生、体育事业的支持，投入了大量财政资金。教育振兴行动计划进展顺利，特别是对贫困地区农村义务教育的支持。公共卫生事业发展，特别是疾病预防控制体系和突发公共卫生事件医疗救治体系的建设。文化体制改革和文化事业发展，加强文化市场管理。加强国土资源、环境保护和生态建设工作。努力改善人民生活，重视解决关系群众切身利益的突出问题。加强民主法制建设，全力维护社会稳定。深入开展廉政建设和反腐败斗争。民族、宗教、侨务工作进一步加强。国防和军队现代化建设取得新进展。外交工作开创新局面，我国的国际地位进一步提高。',
 '这篇文章主要强调了过去一年中国在各方面取得的显著成就，这些成就是由胡锦涛同志为总书记的党中央的领导，全国各族人民的共同努力，以及各地方各部门的艰苦工作共同实现的。文章还对全国各族人民、各民主党派、各人民团体和各界人士对政府工作的信任和支持表示了感谢。文章强调了坚持科学发展观、加强和改善宏观调控、推进改革开放、处理好全局和局部的关系、按客观规律办事、把人民群众利益放在第一位等六个主要体会。同时，文章也指出了经济社会发展中存在的问题和困难，包括经济运行中的突出矛盾、社会发展中的问题、经济社会发展中的长期性问题和深层次矛盾等。最后，文章指出政府工作还存在不少缺点，需要进一步改革和提高效率。',
 '这篇文章主要讨论了2005年中国政府的工作总体部署和经济发展目标。文章强调了要高度重视并解决当前面临的问题，增强责任感，坚持不懈地做好工作。2005年是全面完成“十五”任务、为“十一五”发展打好基础的关键一年。政府工作的基本思路是以邓小平理论

In [61]:
df_sample = pd.read_csv(sample_path)

In [63]:
df_sample['topic'] = df_sample.apply(lambda row: get_topic(row['TITLE'], row['RAW_CONTENT']), axis=1)

In [64]:
df_sample

,ID,RAW_CONTENT,MEETING_NAME,TITLE,PUBLISH_DATE,topic
0,m-941334512-1560944340,李克强主持召开国务院常务会议\r\n部署推进城镇老旧小区改造 顺应群众期盼改善居住条件\r\...,国务院常委会,李克强主持召开国务院常务会议部署推进城镇老旧小区改造 顺应群众期盼改善居住条件确定提前完成农...,2019-06-19T19:39:00Z,基建
1,m-221628530-1512033180,\r\n 中共中央政治局11月30日召开会议，审议通过《中国共产党党务公开条例（试行）》。...,政治局会议,审议通过中国共产党党务公开条例试行,2017-11-30T17:13:00Z,政治
2,m-2186953670-1504012260,中共中央总书记、国家主席、中央军委主席、中央全面深化改革领导小组组长习近平8月29日下午...,中央全面深化改革会议,会议审议通过了关于完善主体功能区战略和制度的若干意见关于探索建立涉农资金统筹整合长效机制的意...,2017-08-29T21:11:00Z,区域发展
3,m-1244349627-1621601580,中共中央总书记、国家主席、中央军委主席、中央全面深化改革委员会主任习近平5月21日下午主...,中央全面深化改革会议,审议通过了关于完善科技成果评价机制的指导意见关于进一步减轻义务教育阶段学生作业负担和校外培训...,2021-05-21T20:53:00Z,"'科技', '教育', '医药健康', '绿色发展', '文化旅游'"
4,m-465612500-1661856840,中共中央政治局8月30日召开会议，研究中国共产党第十九届中央委员会第七次全体会议和中国共...,政治局会议,研究中国共产党第十九届中央委员会第七次全体会议和中国共产党第二十次全国代表大会筹备工作,2022-08-30T18:54:00Z,政治
5,m-594424299-1640240040,李克强主持召开国务院常务会议\r\n确定跨周期调节措施 推动外贸稳定发展\r\n部署做好《区...,国务院常委会,李克强主持召开国务院常务会议确定跨周期调节措施 推动外贸稳定发展部署做好区域全面经济伙伴关系...,2021-12-23T14:14:00Z,出口贸易
6,m-931626733-1609760220,李克强主持召开国务院常务会议\r\n听取《优化营商环境条例》实施情况第三方评估汇报 要求进一...,国务院常委会,李克强主持召开国务院常务会议听取优化营商环境条例实施情况第三方评估汇报 要求进一步打通落实堵...,2021-01-04T19:37:00Z,企业发展
7,m-754197871-1647866220,李克强主持召开国务院常务会议\r\n确定实施大规模增值税留抵退税的政策安排 为稳定宏观经济大...,国务院常委会,李克强主持召开国务院常务会议确定实施大规模增值税留抵退税的政策安排 为稳定宏观经济大盘提供强...,2022-03-21T20:37:00Z,宏观
8,m-75739182-1677945600,（七）坚定扩大对外开放，深化互利共赢的国际经贸合作。面对外部环境变化，实行更加积极主动的开放...,历年政府工作报告,2023年政府工作报告,2023-03-05T00:00:00Z,'宏观'
9,m-780375500-1644929160,李克强主持召开国务院常务会议\r\n听取2021年全国两会建议提案办理情况汇报 要求汇聚众智...,国务院常委会,李克强主持召开国务院常务会议听取2021年全国两会建议提案办理情况汇报 要求汇聚众智做好今年...,2022-02-15T20:46:00Z,'宏观'


In [67]:
df_sample.to_csv('data/sample_output.csv', index = False, encoding='utf_8_sig')

In [68]:
content[:1000]

'\u3000\u3000各位代表：\r\n\u3000\u3000现在，我代表国务院，向大会作政府工作报告，请予审议，并请全国政协各位委员提出意见。\r\n\u3000\u3000一、过去一年工作回顾\r\n\u3000\u30002004年，全国各族人民在中国共产党领导下，高举邓小平理论和“三个代表”重要思想伟大旗帜，团结奋进，锐意创新，社会主义现代化事业取得令人鼓舞的重大成就。\r\n\u3000\u3000过去一年，我们是在应对新的挑战和考验中前进的。近两年我国经济运行中出现了一些新问题，主要是粮食供求关系趋紧，固定资产投资膨胀，货币信贷投放过快，煤电油运紧张。如果任其发展下去，局部性问题就会演变为全局性问题。党中央、国务院审时度势，及时作出了加强宏观调控的决策和部署，按照果断有力、适时适度、区别对待、注重实效的原则，综合运用经济、法律手段和必要的行政手段，着力解决影响经济平稳较快发展的突出问题。经过全国上下共同努力，宏观调控取得明显成效。经济运行中不稳定不健康因素得到抑制，薄弱环节得到加强，避免了经济大的起落。\r\n\u3000\u3000一年来的成就集中表现在：经济保持平稳较快发展，综合国力进一步增强；改革取得重要进展，对外开放实现新突破；社会事业加快发展，人民生活继续改善。2004年，国内生产总值达到13．65万亿元，比上年增长9．5％；财政收入2．63万亿元，增长21．4％；社会消费品零售总额5．4万亿元，增长13．3％；进出口贸易总额1．15万亿美元，增长35．7％，由上年居世界第四位上升为第三位；城镇新增就业980万人，超过预期目标；城镇居民人均可支配收入9422元，农村居民人均纯收入2936元，扣除价格因素，分别实际增长7．7％和6．8％。这些成就，标志着我国在全面建设小康社会道路上又迈出坚实的一步。\r\n\u3000\u3000一年来，我们主要做了以下几方面工作。\r\n\u3000\u3000（一）采取更直接、更有力的政策措施，促进粮食增产和农民增收。\r\n\u3000\u3000这次宏观调控是从加强农业入手的。主要措施是：减免农业税，取消除烟叶以外的农业特产税，对种粮农民实行直接补贴，对部分地区农民实行良种补贴和农机具购置补贴，对重点粮食品种实行最低收购价政策。大幅度增加对农业、农村特别是粮食主产区的建设投入。全年中央财政用于“三农

In [78]:
para = content.split('\r\n\u3000\u3000')

In [79]:
len(para)

78

In [80]:
para[:10]

['\u3000\u3000各位代表：',
 '现在，我代表国务院，向大会作政府工作报告，请予审议，并请全国政协各位委员提出意见。',
 '一、过去一年工作回顾',
 '2004年，全国各族人民在中国共产党领导下，高举邓小平理论和“三个代表”重要思想伟大旗帜，团结奋进，锐意创新，社会主义现代化事业取得令人鼓舞的重大成就。',
 '过去一年，我们是在应对新的挑战和考验中前进的。近两年我国经济运行中出现了一些新问题，主要是粮食供求关系趋紧，固定资产投资膨胀，货币信贷投放过快，煤电油运紧张。如果任其发展下去，局部性问题就会演变为全局性问题。党中央、国务院审时度势，及时作出了加强宏观调控的决策和部署，按照果断有力、适时适度、区别对待、注重实效的原则，综合运用经济、法律手段和必要的行政手段，着力解决影响经济平稳较快发展的突出问题。经过全国上下共同努力，宏观调控取得明显成效。经济运行中不稳定不健康因素得到抑制，薄弱环节得到加强，避免了经济大的起落。',
 '一年来的成就集中表现在：经济保持平稳较快发展，综合国力进一步增强；改革取得重要进展，对外开放实现新突破；社会事业加快发展，人民生活继续改善。2004年，国内生产总值达到13．65万亿元，比上年增长9．5％；财政收入2．63万亿元，增长21．4％；社会消费品零售总额5．4万亿元，增长13．3％；进出口贸易总额1．15万亿美元，增长35．7％，由上年居世界第四位上升为第三位；城镇新增就业980万人，超过预期目标；城镇居民人均可支配收入9422元，农村居民人均纯收入2936元，扣除价格因素，分别实际增长7．7％和6．8％。这些成就，标志着我国在全面建设小康社会道路上又迈出坚实的一步。',
 '一年来，我们主要做了以下几方面工作。',
 '（一）采取更直接、更有力的政策措施，促进粮食增产和农民增收。',
 '这次宏观调控是从加强农业入手的。主要措施是：减免农业税，取消除烟叶以外的农业特产税，对种粮农民实行直接补贴，对部分地区农民实行良种补贴和农机具购置补贴，对重点粮食品种实行最低收购价政策。大幅度增加对农业、农村特别是粮食主产区的建设投入。全年中央财政用于“三农”的支出共2626亿元，增长22．5％。同时，推进农业结构调整，多渠道增加农民收入。这些政策措施力度之大，农民得到实惠之多，是多年来少有的，极大地调动了广大农民的积极性。粮食

In [81]:
len(para[2]), len(para[1]), len(para[8])

(10, 41, 280)

In [82]:
para[1].strip()

'现在，我代表国务院，向大会作政府工作报告，请予审议，并请全国政协各位委员提出意见。'

## Get topic for each paragraph. Ignore short ones.

In [92]:
id_list = []
title_list = []
content_list = []
topic_list = []
ln_min = 50  
cnt = 0

In [93]:
for index, row in df_sample.iterrows():
    txt = re.split('\u3000\u3000|\r\n', row['RAW_CONTENT'])
    txt = [x.strip() for x in txt]
    for x in txt:
        if len(x) >= ln_min:
            id_list.append(row['ID'])
            title_list.append(row['TITLE'])
            content_list.append(x)
            topic_list.append(llm_chain.predict(topic_pdf = topics_pdf, title = row['TITLE'], content = x))
            cnt += 1
            if cnt % 100 == 0:
                print(str(cnt) + ' paragraphs have been processed...')

100 paragraphs have been processed...
200 paragraphs have been processed...
300 paragraphs have been processed...
400 paragraphs have been processed...


In [94]:
len(id_list)

497

In [95]:
df_sample_para = pd.DataFrame({'ID': id_list, 
                               'TITLE': title_list,
                               'CONTENT': content_list,
                              'TOPIC': topic_list})

In [96]:
df_sample_para.head()

,ID,TITLE,CONTENT,TOPIC
0,m-941334512-1560944340,李克强主持召开国务院常务会议部署推进城镇老旧小区改造 顺应群众期盼改善居住条件确定提前完成农...,国务院总理李克强6月19日主持召开国务院常务会议，部署推进城镇老旧小区改造，顺应群众期盼改善...,"基建, 农业农村, 民生扶贫"
1,m-941334512-1560944340,李克强主持召开国务院常务会议部署推进城镇老旧小区改造 顺应群众期盼改善居住条件确定提前完成农...,会议指出，按照中央经济工作会议和《政府工作报告》部署，积极做好“六稳”工作，稳投资是重要方面...,基建
2,m-941334512-1560944340,李克强主持召开国务院常务会议部署推进城镇老旧小区改造 顺应群众期盼改善居住条件确定提前完成农...,会议认为，加快改造城镇老旧小区，群众愿望强烈，是重大民生工程和发展工程。据各地初步摸查，目前...,房地产
3,m-941334512-1560944340,李克强主持召开国务院常务会议部署推进城镇老旧小区改造 顺应群众期盼改善居住条件确定提前完成农...,会议指出，新一轮农村电网改造升级工程自2016年实施以来，完成了农村机井通电、贫困村通动力电...,农业农村
4,m-941334512-1560944340,李克强主持召开国务院常务会议部署推进城镇老旧小区改造 顺应群众期盼改善居住条件确定提前完成农...,会议指出，通过实施农村供水工程，农村居民饮水已提前实现联合国千年发展目标，农村集中供水率、自...,农业农村


In [97]:
df_sample_para.to_csv('data/sample_para.csv', index = False, encoding='utf_8_sig')